In [ ]:
%matplotlib widget
import numpy as np
import matplotlib.pyplot as plt
from scipy.stats import linregress
from tqdm import tqdm

import sys
import os

module_path = os.path.abspath(os.path.join('..'))
if module_path not in sys.path:
    sys.path.append(module_path)

from ews_analysis import hopf_helper as hh
from ews_analysis.saddlenode_ews import dSdt as sd_model
from ews_analysis.pitchfork_helper import *
from ews_helper import get_ews, itoEulerMaruyama

from scipy.integrate import odeint, solve_ivp


In [ ]:
def plot_bif_sim(fig, axs, time, results, derivatives):

    if not isinstance(axs, np.ndarray):
        axs = np.array([axs])
    
    axs[0].plot(time, results[:,0])
    axs[0].set_xlabel('Time')
    # make ax grey
    axs[0].set_facecolor(plt.cm.gray(.85))

    # plot t_star, where r(t) = 0 = r0 + epsilon*t 
    t_star = time[np.where(results[:,1] >= 0)[0][0]]
    axs[0].axvline(t_star, c='r', ls='--') #label=f't*={t_star:.2f}'

    # ax.set_ylim(-20, 70)
    # axs[0].legend()
    axs[0].grid()
    return fig, axs, t_star

In [ ]:
f = np.load('tmp_figs/sd_bif_sim.npz')
time=f['time'] 
results=f['results']
derivatives=f['derivatives']

fig, axs = plt.subplots(1)
fig, axs, t_star = plot_bif_sim(fig, axs, time, results, derivatives)

In [ ]:
"""
time = np.arange(0, 9, 0.001)
r0 = -3
x0 = 0.1
a = 80
epsilon = 1.5
sigma = 0.33
"""

win_size = ews_win_size = 301
offset = ews_offset = 101

print(time.shape)

block_idxs, ar1s, decays, vars = get_ews(
    time, results[:,0], win_size=win_size, offset=offset
)

ews_fig, ews_axs = plt.subplots(2,1)

# Plot AR1s
ews_axs[0].plot(time[block_idxs[:len(ar1s)]], ar1s)
ews_axs[0].axvline(
    t_star, color='r', linestyle='--', 
    alpha=0.5, label='x=0'
)

# Plot Decays
ews_axs[1].plot(time[block_idxs[:len(decays)]], decays)
ews_axs[1].axvline(
    t_star, color='r', linestyle='--', 
    alpha=0.5, label='x=0'
)